# Song Lyrics Collection

In this lesson, we're going to use a Python package called [LyricsGenius](https://github.com/johnwmillr/LyricsGenius) to collect song lyrics about any artist and album.

## API Wrappers
An API wrapper is a package that makes an API easier to use and/or extends the API's functionality. A data scientist named John Miller wrote a Python package called [LyricsGenius,](https://github.com/johnwmillr/LyricsGenius) which makes working with the Genius API easier and adds functionality not offered by the Genius API.

Remember when I said that companies typically don't offer access to their most lucrative data? Well, the Genius API doesn't offer you a way to get access to song lyrics. To solve this problem, LyricsGenius combines the Genius API with the web scraping library BeautifulSoup to get and save song lyrics.

## Install the Package

To install LyricsGenius (and get the most updated version from GitHub), run:

In [ ]:
!pip install git+https://github.com/johnwmillr/LyricsGenius.git

Copy and paste your Genius "Client Access Token" into the quotation marks below, and run the cell to save your variable :

In [11]:
client_access_token = "INSERT YOUR CLIENT ACCESS TOKEN IN THESE QUOTATION MARKS"

Import and initialize LyricsGenius

In [5]:
import lyricsgenius
LyricsGenius = lyricsgenius.Genius(client_access_token)

## Get Songs and Lyrics By a Specific Artist

To get the top songs and song lyrics from a specific artist you can use the method `.search_artist()`:

In [99]:
artist = LyricsGenius.search_artist("Missy Elliott", max_songs=6)

Searching for songs by Missy Elliott...

Song 1: "Work It"
Song 2: "WTF (Where They From)"
Song 3: "Get Ur Freak On"
Song 4: "I’m Better"
Song 5: "Gossip Folks"
Song 6: "The Rain (Supa Dupa Fly)"

Reached user-specified song limit (6).
Done. Found 6 songs.


To access the song titles, you can run `artist.songs`:

In [361]:
artist.songs

[('Work It', 'Missy Elliott'),
 ('WTF (Where They From)', 'Missy Elliott'),
 ('Get Ur Freak On', 'Missy Elliott'),
 ('I’m Better', 'Missy Elliott'),
 ('Gossip Folks', 'Missy Elliott'),
 ('The Rain (Supa Dupa Fly)', 'Missy Elliott'),
 ('Lose Control', 'Missy Elliott'),
 ('One Minute Man', 'Missy Elliott'),
 ('Pass That Dutch', 'Missy Elliott'),
 ('Hot Boyz (Remix)', 'Missy Elliott')]

Inside each of those songs, LyricsGenius has already saved the song lyrics. You can access these lyrics by looping through `artist.songs` and pulling out `song.lyrics`:

In [100]:
for song in artist.songs:
    print(song.lyrics)

[Intro]
DJ, please pick up your phone, I'm on the request line
This is a Missy Elliott one-time exclusive, come on

[Chorus]
Is it worth it? Let me work it
I put my thing down, flip it and reverse it
Ti esrever dna ti pilf, nwod gniht ym tup
Ti esrever dna ti pilf, nwod gniht ym tup
If you got a big *elephant trumpet*, let me search ya
And find out how hard I gotta work ya
Ti esrever dna ti pilf, nwod gniht ym tup
Ti esrever dna ti pilf, nwod gniht ym tup
C'mon

[Verse 1]
I'd like to get to know ya so I could show ya
Put the pussy on ya like I told ya
Give me all your numbers so I can phone ya
Your girl acting stank, then call me over
Not on the bed, lay me on your sofa
Call before you come, I need to shave my chocha
You do or you don't or you will or won't ya?
Go downtown and eat it like a vulture
See my hips and my tips, don't ya?
See my ass and my lips, don't ya?
Lost a few pounds and my waist for ya
This the kinda beat that go ra-ta-ta
Ra-ta-ta-ta-ta-ta-ta-ta-ta-ta
Sex me so good I

## Get Specific Song and Lyrics By a Specific Artist

To get the song lyrics from a specific artist, you can use the method `.search_song()`

In [101]:
song = LyricsGenius.search_song("Missy Elliott", "Work It")

Searching for "Missy Elliott" by Work It...
Done.


In [102]:
song.lyrics

"[Intro]\nDJ, please pick up your phone, I'm on the request line\nThis is a Missy Elliott one-time exclusive, come on\n\n[Chorus]\nIs it worth it? Let me work it\nI put my thing down, flip it and reverse it\nTi esrever dna ti pilf, nwod gniht ym tup\nTi esrever dna ti pilf, nwod gniht ym tup\nIf you got a big *elephant trumpet*, let me search ya\nAnd find out how hard I gotta work ya\nTi esrever dna ti pilf, nwod gniht ym tup\nTi esrever dna ti pilf, nwod gniht ym tup\nC'mon\n\n[Verse 1]\nI'd like to get to know ya so I could show ya\nPut the pussy on ya like I told ya\nGive me all your numbers so I can phone ya\nYour girl acting stank, then call me over\nNot on the bed, lay me on your sofa\nCall before you come, I need to shave my chocha\nYou do or you don't or you will or won't ya?\nGo downtown and eat it like a vulture\nSee my hips and my tips, don't ya?\nSee my ass and my lips, don't ya?\nLost a few pounds and my waist for ya\nThis the kinda beat that go ra-ta-ta\nRa-ta-ta-ta-ta-ta

## Save Lyrics to .txt File

In [103]:
song.save_lyrics(extension='txt')

Wrote Work It to lyrics_missyelliott_workit.txt.


## Get Songs and Lyrics For a Specific Album

As you can see, LyricsGenius is an extremely useful Python package! But one thing that we can't do with LyricsGenius is get all the song lyrics for a particular album.

So we're going to use the web scraping functions that we wrote in the last lesson to get all the song titles for a specific album, then use LyricsGenius to get the lyrics for each of those songs, and then save them all as text files in a directory.

**Import necessary Python libraries**

In [14]:
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
from pathlib import Path

**Make RegEx Function To Clean Up Songs**

In [16]:
def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    return clean_song_title

**Make Function To Scrape Song Titles For Album**

In [8]:
def get_all_songs_from_album(artist, album_name):
    
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")
    
    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
        
    return clean_songs

**Make Function To Download Lyrics For All Songs in Album**

In [18]:
def download_album_lyrics(artist, album_name): 
    
    # Set up LyricsGenius with your Genius API client access token
    #client_access_token = Your-Client-Access-Token
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    # With the function that we previously created, go to Genius.com and get all song titles for a particular artist's album
    clean_songs = get_all_songs_from_album(artist, album_name)
    
    for song in clean_songs:
        
        #For each song in the list, search for that song with LyricsGenius
        song_object = LyricsGenius.search_song(song, artist)
        
        #If the song is not empty
        if song_object != None:
            
            #Do some cleaning and prep for the filename of the song
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            
            #Establish the filename for each song inside a directory that begins with the artist's name and album title
            custom_filename=f"{artist_title}_{album_title}/{song_title}"
            
            #A line of code that we need to create a directory
             #os.makedirs(os.path.dirname(filename), exist_ok=True)
            Path(f"{artist_title}_{album_title}").mkdir(parents=True, exist_ok=True)
            
            #Save the lyrics for the song as a text file
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        
        #If the song doesn't contain lyrics
        else:
            print('No lyrics')

**Call the function**

In [19]:
download_album_lyrics("Missy Elliott", "Under Construction")

Searching for "Intro-Go To The Floor" by Missy Elliott...
Done.
Wrote Intro/Go To The Floor to Missy-Elliott_Under-Construction/Intro-Go-To-The-Floor.txt.
Searching for "Bring the Pain" by Missy Elliott...
Done.
Wrote Bring the Pain to Missy-Elliott_Under-Construction/Bring-the-Pain.txt.
Searching for "Gossip Folks" by Missy Elliott...
Done.
Wrote Gossip Folks to Missy-Elliott_Under-Construction/Gossip-Folks.txt.
Searching for "Work It" by Missy Elliott...
Done.
Wrote Work It to Missy-Elliott_Under-Construction/Work-It.txt.
Searching for "Back in the Day" by Missy Elliott...
Done.
Wrote Back in the Day to Missy-Elliott_Under-Construction/Back-in-the-Day.txt.
Searching for "Funky Fresh Dressed" by Missy Elliott...
Done.
Wrote Funky Fresh Dressed to Missy-Elliott_Under-Construction/Funky-Fresh-Dressed.txt.
Searching for "Pussycat" by Missy Elliott...
Done.
Wrote Pussycat to Missy-Elliott_Under-Construction/Pussycat.txt.
Searching for "Nothing Out There for Me" by Missy Elliott...
Done.
W